In [1]:
import os
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas
import numpy as np
import sys
from evaluation.metric import computeAccuracy, computeF1Score, computeAUC
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline

0.17


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa    = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
# clf_fpl_dl = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pandas.read_csv("./data/alcohol_training_instances.csv")


In [3]:
clf_alc_initial.steps
# with open('structure.txt', 'w') as f:
#     tmp = ''.join(str(e) for e in clf_alc_initial.steps)
#     f.write(tmp)

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=71233, min_df=1,
     ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  LogisticRegression(C=177.08103265690417, class_weight=None, dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='ovr', n_jobs=None, penalty='l2', random_state=None,
            solver='liblinear', tol=0.000655077907893521, verbose=0,
            warm_start=None))]

Prepare the alcohol data

In [4]:
alc_labels = []
# extracting pre-labeled classification
for i in range(0,len(df.labels)):
    alc_labels.append(df.labels[i][12])
alc_labels = list(map(int, alc_labels))
print(alc_labels[:10])
print(len(alc_labels))
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, alc_labels, test_size=0.1, random_state=26)

[0, 1, 1, 1, 1, 1, 1, 1, 0, 1]
15651


First we check the initial model performance

In [5]:
clf_alc_initial.fit(X_train, y_train)
# predict in testing set.
y_alc_initial = clf_alc_initial.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_initial, y_test)
f1 = computeF1Score(y_test, y_alc_initial)
print('F1 Score: ', f1)
auc_score_initial = computeAUC(clf_alc_initial, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_initial)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1566
pretrained alcohol classifer:  261 different item in whole dataset
accuracy: 83.333333%
F1 Score:  0.8718703976435936
alcohol LR AUC:  0.8722371835589783


Then we make our own RF model and update structure parameters in RF model. To make sure they become same with the initial model

In [9]:
clf_alc_RF = AlcoholPipeline(global_features=["text"]).pipeline(RandomForestClassifier())
params_RF = {'features__text__tfidf__dtype':np.int64, 'features__text__tfidf__max_features':71233, 'features__text__tfidf__ngram_range':(2,5) 
            }
# to see all parameters in model
# clf_alc_LR_updateParams.get_params().keys()
clf_alc_RF.set_params(**params_RF)
clf_alc_RF.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=71233, min_df=1,
     ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False))]

Train the SVM model (with updated parameters), and predict in testing set

In [11]:
clf_alc_RF.fit(X_train, y_train)
# predict in testing set.
y_alc_RF = clf_alc_RF.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_alc_RF, y_test)
f1_score_RF = computeF1Score(y_test, y_alc_SVM)
print('F1 Score: ', f1_score_RF)
# from sklearn.metrics import roc_auc_score
# y_score = clf_alc_SVM.predict_proba(X_test)
# roc_auc_score(y_test,y_score[:,1])
# y_score[:,1]
# print(y_score)
auc_score_RF = computeAUC(clf_alc_RF, X_test, y_test, plot=False, RF=True)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol RF AUC: ', auc_score_RF)
clf_alc_RF.steps

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1566
pretrained alcohol classifer:  245 different item in whole dataset
accuracy: 84.355045%
F1 Score:  0.8715462918080465
alcohol RF AUC:  0.8723071781169014


[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=71233, min_df=1,
     ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False))]

We can see the RF model we build with updated parameters achieve almost the same results as the initial model, compared to initial model with 0.87 in F1 score and 0.87 in AUC score.